In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
import os
import requests
df = pd.read_csv('tokopedia_scrape.csv', sep=';')
# Path folder tujuan untuk menyimpan gambar
folder_path = 'gambar_produk'

# Tambahkan kolom baru 'path_gambar' ke dalam DataFrame
df['path_gambar'] = ''

# Iterasi melalui setiap baris dalam DataFrame
for index, row in df.iterrows():
    # Mendapatkan URL gambar dari kolom 'gambar'
    image_url = row['gambar']
    # Mendapatkan nama file dari kolom 'produk' atau indeks baris
    filename =  f"image_{index}.jpg"
    # Menggabungkan path folder dengan nama file
    file_path = os.path.join(folder_path, filename)

    # Mengirim permintaan GET ke URL gambar
    response = requests.get(image_url)
    # Membaca konten gambar
    image_content = response.content

    # Menyimpan gambar ke folder
    with open(file_path, 'wb') as file:
        file.write(image_content)

    # Memperbarui kolom 'path_gambar' dengan path file gambar
    df.at[index, 'path_gambar'] = file_path

    print('Gambar berhasil disimpan:', file_path)

df.to_excel('DataGambarFix.xlsx')


In [66]:
import os
import requests
import pandas as pd
import shutil

df = pd.read_excel('DataGambar.xlsx')

# Iterasi melalui setiap baris dalam DataFrame
for index, row in df.iterrows():
    # Mendapatkan path gambar dari kolom 'path_gambar'
    file_path = row['path_gambar']
    # Mendapatkan nilai kolom 'produk_laku'
    produk_laku = row['produk_laku']

    # Menentukan folder tujuan berdasarkan nilai kolom 'produk_laku'
    if produk_laku == 0:
        target_folder = 'gambar_produk/laku'
    else:
        target_folder = 'gambar_produk/tidak_laku'

    # Membuat folder tujuan jika belum ada
    os.makedirs(target_folder, exist_ok=True)

    # Mendapatkan nama file dari path
    filename = os.path.basename(file_path)

    # Menggabungkan path folder tujuan dengan nama file
    destination_path = os.path.join(target_folder, filename)

    # Memindahkan file ke folder tujuan
    shutil.move(file_path, destination_path)

    # Mengubah path menjadi format yang sesuai dengan sistem operasi
    new_path = os.path.normpath(destination_path)

    print('Gambar berhasil dipindahkan:', destination_path)

    # Memperbarui kolom 'path_gambar' dengan path file gambar
    df.at[index, 'path_gambar'] = new_path

df.to_excel('DataGambarFix.xlsx')


Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_0.jpg
Gambar berhasil dipindahkan: gambar_produk/laku\image_1.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_2.jpg
Gambar berhasil dipindahkan: gambar_produk/laku\image_3.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_4.jpg
Gambar berhasil dipindahkan: gambar_produk/laku\image_5.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_6.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_7.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_8.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_9.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_10.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_11.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_12.jpg
Gambar berhasil dipindahkan: gambar_produk/laku\image_13.jpg
Gambar berhasil dipindahkan: gambar_produk/tidak_laku\image_14.jpg
Gambar berhasil dipindahkan: 

In [26]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Conv2D, Flatten, Concatenate

# Define image data generator
image_data_generator = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values to [0, 1]

# Load and preprocess image data from folder
image_data = image_data_generator.flow_from_directory(
    'gambar_produk/',
    target_size=(64, 64),
    batch_size=10,
    class_mode='categorical'
)

# Define image input shape based on loaded image data
image_input_shape = image_data.image_shape
image_input = Input(shape=image_input_shape)

Found 6000 images belonging to 2 classes.


In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.feature import hog

# Mendefinisikan path ke folder gambar
path_gambar = 'gambar_produk'

# Mendefinisikan list kelas
kelas = ['laku', 'tidak_laku']

# Menginisialisasi list untuk menyimpan gambar dan labelnya
gambar = []
label = []
gambar_path = []

# Memuat gambar dari setiap subfolder
for i, k in enumerate(kelas):
    subfolder_path = os.path.join(path_gambar, k)
    for filename in os.listdir(subfolder_path):
        img_path = os.path.join(subfolder_path, filename)
        gambar_path.append(img_path)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Mengubah ke citra keabuan
        img = cv2.resize(img, (128, 128))  # Mengubah ukuran gambar sesuai kebutuhan

        # Ekstraksi fitur menggunakan HOG
        features = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

        gambar.append(features)
        label.append(i)  # Memberikan label berdasarkan indeks kelas

# Mengonversi list menjadi array numpy
gambar = np.array(gambar)
label = np.array(label)

In [24]:
testing = pd.read_csv('tokopedia_scrape.csv', sep=';')
testing['bahan'].value_counts()

<IPython.core.display.Javascript object>

plastik    4105
Plastik    1055
Besi        283
Kain        180
PLASTIK     171
Kaca        120
KAIN         52
kain         34
Name: bahan, dtype: int64

In [2]:
df_fix = pd.DataFrame(gambar)
df_fix['label'] = label
df_fix['path_gambar'] = gambar_path
df_fix.tail()

<IPython.core.display.Javascript object>

,0,1,2,3,4,5,6,7,8,9,...,8092,8093,8094,8095,8096,8097,8098,8099,label,path_gambar
5995,0.345631,0.04693,0.189574,0.007272,0.379794,0.018595,0.006006,0.001557,0.114665,0.099131,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1,gambar_produk\tidak_laku\image_994.jpg
5996,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1,gambar_produk\tidak_laku\image_995.jpg
5997,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.563050,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1,gambar_produk\tidak_laku\image_996.jpg
5998,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1,gambar_produk\tidak_laku\image_997.jpg
5999,0.108790,0.04507,0.030680,0.014368,0.043674,0.004410,0.011500,0.030061,0.046959,0.176838,...,0.049434,0.056722,0.080726,0.10835,0.096881,0.056878,0.092126,0.065339,1,gambar_produk\tidak_laku\image_999.jpg


In [3]:
data = pd.read_excel('DataGambarFix.xlsx')
data = data[['harga', 'susunan', 'bahan', 'jumlah_terjual', 'produk_laku', 'path_gambar']]
fix_df = pd.merge(df_fix, data, on='path_gambar')  # Replace 'common_column' with the actual common column name
fix_df = fix_df.drop(['path_gambar', 'jumlah_terjual', 'produk_laku'], axis = 1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
fix_df

,0,1,2,3,4,5,6,7,8,9,...,8094,8095,8096,8097,8098,8099,label,harga,susunan,bahan
0,0.012029,0.000000,0.001791,0.000000,0.000633,0.000000,0.002686,0.000000,0.000000,0.528610,...,0.000000,0.002466,0.000000,0.003487,0.000000,0.000000,0,269500,3,3
1,0.257562,0.000000,0.295896,0.000000,0.257562,0.000000,0.000000,0.000000,0.000000,0.154537,...,0.019765,0.682977,0.000000,0.006250,0.000000,0.000000,0,34000,3,3
2,0.184802,0.083545,0.189024,0.010166,0.103131,0.024216,0.196267,0.092986,0.145412,0.016272,...,0.100828,0.144879,0.020135,0.004149,0.073484,0.043730,0,698250,3,3
3,0.143932,0.081881,0.185941,0.000000,0.137278,0.007024,0.200843,0.094592,0.098530,0.034334,...,0.106549,0.131244,0.064661,0.000000,0.022666,0.000000,0,523000,3,3
4,0.147263,0.087672,0.193099,0.012116,0.130256,0.024613,0.196807,0.080732,0.089406,0.029464,...,0.051728,0.196661,0.008682,0.082914,0.004110,0.041292,0,1498000,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.345631,0.046930,0.189574,0.007272,0.379794,0.018595,0.006006,0.001557,0.114665,0.099131,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1740000,3,0
5996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,299000,3,2
5997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.563050,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,276000,5,3
5998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,228720,3,3


In [4]:
fix_df["bahan"] = fix_df["bahan"].str.lower()
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
# Melabelkan target menggunakan LabelEncoder
label_encoder = LabelEncoder()
fix_df['bahan'] = label_encoder.fit_transform(fix_df['bahan'])

In [5]:
X = fix_df.drop('label', axis=1).values
Y = fix_df['label'].values

In [25]:
from imblearn.combine import SMOTETomek
smotetomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smotetomek.fit_resample(X, Y)

In [29]:
a = pd.DataFrame(y_resampled)
a.value_counts()

<IPython.core.display.Javascript object>

0    5446
1    5446
dtype: int64

In [30]:
x_train_ml, x_test_ml, y_train_ml, y_test_ml = train_test_split(X_resampled, y_resampled, random_state=42, test_size=0.20, shuffle=True)
x_train_ml.shape, y_train_ml.shape, x_test_ml.shape, y_test_ml.shape

((8713, 8103), (8713,), (2179, 8103), (2179,))

In [31]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()

y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [32]:
x_train_dl, x_test_dl, y_train_dl, y_test_dl = train_test_split(X, y, random_state=42, test_size=0.20, shuffle=True)
x_train_dl.shape, y_train_dl.shape, x_test_dl.shape, y_test_dl.shape

((4800, 8103), (4800, 2), (1200, 8103), (1200, 2))

In [33]:
scaler = StandardScaler()

x_train_ml = scaler.fit_transform(x_train_ml)
x_test_ml = scaler.transform(x_test_ml)

x_train_dl = scaler.fit_transform(x_train_dl)
x_test_dl = scaler.transform(x_test_dl)

In [34]:
x_train_dl = np.expand_dims(x_train_dl, axis=2)
x_test_dl = np.expand_dims(x_test_dl, axis=2)
x_train_dl.shape, y_train_dl.shape , x_test_dl.shape , y_test_dl.shape

((4800, 8103, 1), (4800, 2), (1200, 8103, 1), (1200, 2))

In [35]:
# parameter for tuning
params={
    "learning_rate":[0.05,0.10,0.15],
    "max_depth":[3,5,7,9],
    "min_child_weight":[5,7, 9],
    "gamma":[0.5,0.1,0.3],
    "colsample_bytree":[0.3,0.5,0.7],
    "random_state":[1970, 2021, 2020]
}

In [36]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
# use xgboost hyperparameter tuning with method gpu
classifier=xgb.XGBClassifier(tree_method='hist')

In [37]:
# hyperparameter tuning with cross validation 5 fold
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='f1_macro',cv=5,verbose=3)
random_search.fit(x_train_ml,y_train_ml)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, random_state=2021;, score=1.000 total time=  11.9s
[CV 2/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, random_state=2021;, score=1.000 total time=  11.5s
[CV 3/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, random_state=2021;, score=1.000 total time=  11.5s
[CV 4/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, random_state=2021;, score=1.000 total time=  13.2s
[CV 5/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, random_state=2021;, score=1.000 total time=  11.5s
[CV 1/5] END colsample_bytree=0.5, gamma=0.5, learning_rate=0.15, max_depth=9, min_child_weight=9, random_state=2021;, score=1.000 total time=  11.0s
[CV 2/5] END colsample_bytree=0.5, gamma

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           min_child_weight=None, missing=nan,
                

In [38]:
# best parameter
random_search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=2021, ...)

In [39]:
# modelling with best estimator
xgbt = random_search.best_estimator_
xgbt.fit(x_train_ml,y_train_ml)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=2021, ...)

In [40]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# predict to data test
Y_pred = xgbt.predict(x_test_ml)
Y_pred_tr = xgbt.predict(x_train_ml)
# confusion matrix
cnf_matrix = confusion_matrix(y_test_ml, Y_pred)
print('Akurasi test model:', accuracy_score(y_test_ml, Y_pred))
print()
print("Confusion Matrix")
print(cnf_matrix)
print()
print(classification_report(y_test_ml, Y_pred))

Akurasi test model: 1.0

Confusion Matrix
[[1069    0]
 [   0 1110]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1069
           1       1.00      1.00      1.00      1110

    accuracy                           1.00      2179
   macro avg       1.00      1.00      1.00      2179
weighted avg       1.00      1.00      1.00      2179



In [41]:
import pickle

# Assuming you have trained a model named 'model'
# Save the model to a pickle file
with open('model_xgboost_31.pkl', 'wb') as file:
    pickle.dump(xgbt, file)